# 3 Preprocessing

**Prerequisites:** master data and transaction data has been cleaned & processed

## Library Imports

In [1]:
import pandas as pd
import numpy as np

## 3.1 Import raw data
master & transaction data for combining the data

In [2]:
df_master = pd.read_csv("data new/masterdata.csv",encoding="utf-8",sep=",",usecols=[0,1],dtype={"CardID":object})
print("Data entries masterdata: " + str(len(df_master.index)))

Data entries masterdata: 75


In [3]:
df_trans = pd.read_csv("data new/transdata.csv",encoding="utf-8",sep=",",usecols=[0,1,6,7,8,9,10],parse_dates=["BookingDate"],dtype={"CardID":object})
print("Data entries transdata: " + str(len(df_trans.index)))

Data entries transdata: 950


## 3.2 Merge dataframes
Additionally a short overview about the merged data is given.

In [4]:
df_merged = pd.merge(df_master, df_trans, how="outer", on="CardID", indicator=True)
df_merged = df_merged.sort_values(["CardID","BookingDate"], ascending=[True,True])

In [5]:
print("Data entries: " + str(len(df_merged.index)))

#CardID who are not in masterdata
#Reasons: wrong manual adaption, encoding error in transdata or missing card entry in masterdatabase
print("Transactions with CardID not in masterdata: -" + str(len(df_merged.loc[df_merged["_merge"] == "right_only"])))

print("Customers without transactions: -" + str(len(df_merged.loc[df_merged["_merge"] == "left_only"])))
print("Merged transdata which can be used: " + str(len(df_merged.loc[df_merged["_merge"] == "both"])))

Data entries: 950
Transactions with CardID not in masterdata: -0
Customers without transactions: -0
Merged transdata which can be used: 950


## 3.3 Data Cleaning
### 3.3.1 Customers with no transactions
Filtering of customers who have not purchased in the evaluation period (transtype = NaN) and exporting a list of those costomers.

In [6]:
df_cust0 = df_merged.loc[df_merged["_merge"] == "left_only"]

df_cust0.to_csv("data excluded/excluded 5_customers without transaction.csv", encoding="utf-8", sep=",")
print("Customers without transaction: " + str(len(df_cust0.index)))                       

Customers without transaction: 0


In [7]:
#Cleaning the data
df_merged = df_merged.loc[~(df_merged["_merge"] == "left_only")]
print("Data entries after cleaning 1: " + str(len(df_merged.index)))

Data entries after cleaning 1: 950


### 3.3.2 Transactions with missing customer link
Filtering of transactions which are not linked to a data entry in masterdata.

In [8]:
df_missingcustid = df_merged.loc[df_merged["_merge"] == "right_only"]
df_missingcustid.to_csv("data excluded/excluded 6_transactions with missing customer link.csv",encoding="utf-8",sep=",") 

In [9]:
#Cleaning the data
df_merged = df_merged.loc[~(df_merged["_merge"] == "right_only")]
print("Data entries after cleaning 2: " + str(len(df_merged.index)))

Data entries after cleaning 2: 950


## 3.4 Grouping the cleaned data
**Objective:** One collumn should represent a single customer purchase

Rounding the floats on two decimal places and adding price per article an total purchases.

Afterwards the dataframe will be saved.

In [10]:
df_merged["TransactionType"] = df_merged["TransactionType"].astype("int64")
df_merged["Store"] = df_merged["Store"].astype("int64")

df_proc = df_merged.groupby(["CustID","CardID","BookingDate","Store","TransactionType"],
                             as_index=False).agg(np.sum)

In [11]:
df_proc["Revenue"] = np.round(df_proc["Revenue"], decimals=2)
df_proc["RevenueInc."] = np.round(df_proc["RevenueInc."], decimals=2)
df_proc["Price/Article"] = np.round(df_proc["Revenue"].divide(df_proc["Qty"]), decimals=2)

df_proc.to_csv("data new/procdata.csv",encoding = "utf-8",sep = ",",index = False)
print("Data entries: " + str(len(df_proc.index)))

Data entries: 944


In [12]:
df_proc.head()

,CustID,CardID,BookingDate,Store,TransactionType,Revenue,RevenueInc.,Qty,Price/Article
0,1601060367199,59681386599,2017-05-15,4,1,145.63,137.63,4.0,36.41
1,1601060367199,59681386599,2017-06-08,3,1,115.51,115.51,2.0,57.76
2,1601060367199,59681386599,2017-06-23,9,1,15.34,15.34,1.0,15.34
3,1601060367199,59681386599,2017-06-26,7,1,30.67,30.67,2.0,15.34
4,1601060367199,59681386599,2017-07-14,9,1,31.94,31.94,1.0,31.94
